In [1]:
# import data
logfile = open("../../Data/Logs/light1_first_se_log.txt","r")
loglines = logfile.readlines()
print (loglines)

['# Script started\n', 'SE0>rtc read\n', 'curr rtc time 1646297867.353394 s, 2022-03-03 08:57:47 \n', 'OK\n', 'SE0># Script delay (avoiding SAA)\n', 'SE0>script delayuntil 1646297935\n', 'OK\n', 'SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)\n', 'SE0>csp txrx 4 14 600 000500302A\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0008000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0005010100\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007011400\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp t

In [41]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    I = i+1
    l = loglines[I][14:]
    if "SE0>rtc read" in k:
        print ("Line:" , i , ", Command:" , k , ", Time:" , l )

# if the value is in range of the full range of file?
# first rtc must be start of code, second rtc must be end of code


Line: 1 , Command: SE0>rtc read
 , Time: 1646297867.353394 s, 2022-03-03 08:57:47 

Line: 139 , Command: SE0>rtc read
 , Time: 1646299664.765045 s, 2022-03-03 09:27:44 



IndexError: list index out of range

In [32]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    # l = loglines[i+1]
    if k[0:12] in "SE0>rtc read":
        # l=i+1
        print ("line: " , i , ", command: " , k )




    # if k[0:5] in "SE0>#":
    #     print (k)

line:  1 , command:  SE0>rtc read

line:  139 , command:  SE0>rtc read

line:  142 , command:  SE0>


In [ ]:
# Work to find the following:

# "Time,scenario"
# "At time () command () was excuted.
# "At time () command () failed."

# If line in SE0>:
# print the time
# print the command
# print the info
# print if command was excuted or failed